In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import networkx as nx
%matplotlib inline


In [2]:
import cvxpy as cp

In [3]:
plt.rcParams['figure.figsize'] = [15, 10]
sns.set_style('whitegrid')
plt.rcParams['font.size'] = 20.0
plt.rcParams['xtick.labelsize'] = 20.0
plt.rcParams['ytick.labelsize'] = 20.0
import pickle

# Generate synthetic data

In [4]:
n = 20
ground_truth_asymm = np.random.uniform(0, 0.5, (n, n))
ground_truth_mat = ground_truth_asymm + ground_truth_asymm.T

In [5]:
def gen_sample_mat(ground_truth_mat):
    '''
    Generate a sample matrix from a ground truth matrix
    inputs:
        ground_truth_mat: A n by n matrix of probabilities
    outputs:    
        A: a n by n matrix whose entries are 1 
        with probability ground_truth_mat[i, j] and 0 otherwise
    '''
    n = ground_truth_mat.shape[0]
    A = np.zeros_like(ground_truth_mat)
    for i in range(n):
        for j in range(i + 1):
            sample = np.random.binomial(1, ground_truth_mat[i, j])
            A[i, j] = sample
            A[j, i] = sample
    return A

In [6]:
sample_mats = [gen_sample_mat(ground_truth_mat) for _ in range(11)]

scale = np.sum(sample_mats[0])
for i in range(len(sample_mats)):
    if i > 0:
        s = np.sum(sample_mats[i])
        rescale = scale / s 
        sample_mats[i] = rescale * sample_mats[i]

In [7]:
for i in range(11): 
    print(np.sum(sample_mats[i]))

178.0
178.0
178.0
177.99999999999997
178.0
178.0
178.0
177.99999999999997
178.00000000000003
178.0
178.00000000000003


Notice that the sum of the entries of each matrix is the same, and is near 0.5 * 400. 

## Define the convex program

For now, we will just set delta manually to be O(sqrt(degree)). Here, the degree is roughly 10, so we set delta to be sqrt(10).

In [16]:
m = len(sample_mats)
n = sample_mats[0].shape[0]
delta = np.sqrt(10.0)

eta = cp.Variable(m, nonneg=True)
# partial_traces = cp.Variable(n)

# define the Y_diff matrix
X_0 = eta[0] * sample_mats[0]
X_arrs = []
for i in range(1, len(sample_mats)): 
    X_arrs.append(eta[i] * (X_0 - sample_mats[i]))
Y = X_0 + np.sum(X_arrs, axis=0)
# Y_diff = Y - delta * np.eye(n)

ob1 = cp.atoms.lambda_sum_largest(Y, 1) - delta
ob2 = cp.atoms.lambda_sum_largest(Y, 2) - 2 * delta

# objective = cp.Minimize(cp.maximum(ob1, ob2))
ob = cp.atoms.trace(Y)
objective = cp.Minimize(ob)
# zero_mat = np.zeros_like(Y_diff)
# mat_max = cp.atoms.max(Y_diff, zero_mat)
# max_elemwise(Y_diff, zero_mat)

# obj = cp.atoms.lambda_sum_largest(Y_diff, 1)
# for i in range(2, n + 1):
#     obj2 = cp.atoms.lambda_sum_largest(Y_diff, i)
#     obj = cp.atoms.maximum(obj, obj2)

# partial_traces = [[]] * n
# for j in range(1, n + 1): 
#     partial_trace = cp.atoms.lambda_sum_largest(Y_diff, j)
#     partial_traces[j - 1] = partial_trace

# max_traces = cp.atoms.elementwise.maximum(partial_traces)
# objective = cp.Minimize(obj)



# Attempt with BFGS in Scipy

In [8]:
from scipy import *

In [9]:
# # objective function
# def objective(x):
# 	return x[0]**2.0 + x[1]**2.0

# # derivative of the objective function
# def derivative(x):
# 	return [x[0] * 2, x[1] * 2]

# # define range for input
# r_min, r_max = -5.0, 5.0

# ...
# # define the starting point as a random sample from the domain
# pt = r_min + rand(2) * (r_max - r_min)


In [10]:
import scipy

## trying BFGS 

In [28]:
def matrix_lin_combo(eta_arr, sample_mats): 
    X_0 = np.copy(eta_arr[0] * sample_mats[0])
    for i in range(1, len(sample_mats)): 
        X_i = eta_arr[i] * (X_0 - sample_mats[i])
        X_0 += X_i
    return X_0


In [64]:
delta = np.sqrt(10.0)

def objective(eta_arr): 
    eigs = scipy.linalg.eigh(matrix_lin_combo(eta_arr, sample_mats), eigvals_only=True)
    # print('eigs are ', eigs)
    if eigs[-1] < delta: 
        return 0.0
    ob1 = eigs[-1] - delta
    # print('objective is ', ob1)
    return ob1
    # ob2 = sum(eigs[-2:]) - 2 * delta
    # return max(ob1, ob2)

def derivative(eta_arr): 
    eigvals, eigvecs = scipy.linalg.eigh(matrix_lin_combo(eta_arr, sample_mats))
    X_arrs_unscaled = [sample_mats[i] - sample_mats[0] if i !=0 else sample_mats[0] for i in range(len(sample_mats))]

    # list of lists 
    all_quad_forms = np.array([(eigvecs.T.dot(X_arrs_unscaled[i])*eigvecs.T).sum(axis=1) for i in range(len(X_arrs_unscaled))])
    # print(np.array(all_quad_forms).shape)
    # print(all_quad_forms)
    return all_quad_forms[:, 0]

    # der_0 = np.trace(sample_mats[0])
    # ders = [der_0 - np.trace(sample_mats[i]) for i in range(1, len(sample_mats))]
    # return [der_0] + ders

In [49]:
def simplex_constraint(eta_arr): 
    return np.sum(eta_arr) - 1.0

In [50]:
eta_init = np.random.uniform(0, 1, len(sample_mats))
eta_init = eta_init / np.sum(eta_init)
bounds_list = [(0, 1) for _ in range(len(sample_mats))]

In [51]:
cons = {'type':'eq', 'fun': simplex_constraint}

In [53]:
eta_init

array([0.12500584, 0.06135611, 0.0990751 , 0.09554297, 0.02772868,
       0.06314039, 0.03428763, 0.13133985, 0.15146538, 0.09049789,
       0.12056016])

In [52]:
derivative(eta_init)

array([8.54624141, 0.22087045, 0.38283026, 0.53354853, 0.17369917,
       0.34929231, 0.51201137, 0.41772135, 0.39601352, 0.46160701,
       0.54641337])

In [70]:
delta = 1.0

In [71]:
result = scipy.optimize.minimize(
    objective,
    eta_init,
    method='SLSQP',
    jac=derivative,
    bounds=bounds_list,
    constraints=cons, 
    options = {
        'maxiter': 1000,
        'disp': True
    }
)


eigs are  [-9.17688817 -2.26785616 -1.79160634 -1.69029227 -1.52510858 -1.15911197
 -0.53588422 -0.29979455 -0.18490056 -0.12070151  0.05375117  0.29084765
  0.61176211  0.765975    0.99579571  1.04727159  1.41761017  1.44448463
  1.97116263  2.31968036]
objective is  1.319680357969549
eigs are  [-13.53405841  -2.78307126  -2.55112824  -2.37094897  -2.21853939
  -1.44657307  -1.13876156  -0.64016461  -0.57311285  -0.224463
  -0.08519456   0.39638835   0.91283754   1.13903585   1.59708402
   1.6563851    2.19644005   2.38933531   2.55341815   3.23304967]
objective is  2.233049674128843
eigs are  [-1.04680247e+01 -2.38486889e+00 -1.89454292e+00 -1.81636001e+00
 -1.54083415e+00 -1.27003027e+00 -6.10686850e-01 -4.74340143e-01
 -2.48553946e-01 -8.96567386e-02  4.94547493e-03  4.15503080e-01
  8.07965685e-01  8.23155862e-01  9.12489161e-01  1.19202791e+00
  1.47243433e+00  1.60594879e+00  2.16171887e+00  2.42725069e+00]
objective is  1.427250693357235
eigs are  [-9.66218944e+00 -2.30838664e+

In [72]:
result

     fun: 1.3209322435415851
     jac: array([8.54832079, 0.22031599, 0.38123652, 0.5314114 , 0.1733599 ,
       0.347932  , 0.50979325, 0.41600984, 0.39453941, 0.45973728,
       0.54395575])
 message: 'Iteration limit reached'
    nfev: 1079
     nit: 100
    njev: 99
  status: 9
 success: False
       x: array([0.12456415, 0.06169183, 0.0994571 , 0.09520538, 0.02846912,
       0.06344976, 0.03418764, 0.13135279, 0.15100521, 0.09018638,
       0.12043065])

In [73]:
objective(result['x'])

eigs are  [-9.19368138 -2.26920256 -1.79257362 -1.69040686 -1.52559118 -1.16267152
 -0.53656436 -0.30025056 -0.18597015 -0.11780469  0.05330638  0.29048262
  0.61341478  0.7660495   0.99500074  1.04880085  1.41844131  1.4448421
  1.9733721   2.32093224]
objective is  1.3209322435415851


1.3209322435415851

In [74]:
np.linalg.norm(matrix_lin_combo(result['x'], sample_mats))

10.78751178806719

In [75]:
result['x']

array([0.12456415, 0.06169183, 0.0994571 , 0.09520538, 0.02846912,
       0.06344976, 0.03418764, 0.13135279, 0.15100521, 0.09018638,
       0.12043065])

In [41]:
for mat in sample_mats: 
    print(np.linalg.norm(sample_mats[0] - mat))

0.0
12.272545971206814
12.011398095580976
12.42528692232259
12.284667007219237
12.346701002333605
12.362373672894815
12.519835009165114
11.147383143294045
12.094512297846999
12.224722911681052


In [68]:
scipy.linalg.eigh(matrix_lin_combo(result['x'], sample_mats), eigvals_only=True)

array([-9.18119995, -2.26819648, -1.79178283, -1.69027412, -1.52497741,
       -1.15996778, -0.53605877, -0.29984678, -0.18525576, -0.11973005,
        0.05346027,  0.29083033,  0.61218599,  0.76610624,  0.99541947,
        1.04765813,  1.41762519,  1.44465109,  1.97181594,  2.31981495])

In [69]:
matrix_lin_combo(result['x'], sample_mats) - matrix_lin_combo(result['x'], sample_mats).T

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.

Okay, so somehow since the largest eigenvalue is less than our delta, we get a different result. What if we set delta to be very small?

Clearly something is wrong. It should just try to minimize the spectral norm, and yet setting e.g. eta_1 = 1.0 would result in a better output. 

We do know that the result of the optimizer says status:failed. Why is that? 